# Learn Train object
### *Narges Rezaie*
#### Vignette built on Apr 1, 2023 with Topyfic version 0.2.0.

The aim of this tutorials is show you how you can train your data faster!

The most time-consuming part is learning train object. In order to get the trian object faster, you can tune the parameters like number of jobs or number of threads, however if your data is big it's going to spend some time to copy your data. One other way is to submit job simolotinalsy in order to get your train object faster to have one train object per random state and then use `combine_LDA_models()` function to have one train object.

Below you can find the script about how you can make and submit jobs.

**note**: Don't forget to modify the script base on the machine you used.

In [ ]:
#!/bin/sh
#SBATCH -A SEYEDAM_LAB
#SBATCH --cpus-per-task 1
#SBATCH --output=LDAs.out
#SBATCH --error=LDAs.err
#SBATCH --time=1:00:00
#SBATCH -J topyfic_control
#SBATCH --mail-type=START,END
#SBATCH --partition=standard

data=../../data/all_ModelAD_ENCODE_5xFAD_final.h5ad

for i in {0..99}
do
    scriptName=run_${i}
    curr=${scriptName}.sh
    echo '#!/bin/bash' > ${curr}
    echo '#SBATCH -A SEYEDAM_LAB' >> ${curr}
    echo '#SBATCH --cpus-per-task 10' >> ${curr}
    echo '#SBATCH --output=LDA-%J.out' >> ${curr}
    echo '#SBATCH --error=LDA-%J.err' >> ${curr}
    echo '#SBATCH --time=02:00:00' >> ${curr}
    echo '#SBATCH -J topyfic-control-%J' >> ${curr}
    echo '#SBATCH --mail-type=START,END' >> ${curr}
    echo '#SBATCH --partition=standard' >> ${curr}
    
    echo "python3 topyfic_train.py ${data} 5 ${i}" >> ${curr}
    
    chmod +x ${curr}
    sbatch ${scriptName}.sh
    
done

### topyfic_train.py

In [1]:
import sys
import Topyfic
import pandas as pd
import scanpy as sc
import numpy as np
import random

#input
path = str(sys.argv[1])
k = int(sys.argv[2])
random_state = int(sys.argv[3])

data = sc.read_h5ad(path)


train = Topyfic.Train(name=f"train_{k}_{random_state}",
                      k=k,
                      n_runs=1,
                      random_state_range=[random_state])

train.run_LDA_models(data, batch_size=128, max_iter=5, n_jobs=1, n_thread=1)

train.save_train()

### combine all singe LDA runs

one you have all your train runs with one random seed, you can follow the code bellow to make train objects contain all runs you want

**note**: don't forget to modify `k` and `path`.

In [4]:
import Topyfic
import pandas as pd
import scanpy as sc
import numpy as np
import random

k = 5
path = "../../data/all_ModelAD_ENCODE_final.h5ad"

data = sc.read_h5ad(path)

main_train = Topyfic.Train(name=f"train_{k}",
                           k=k,
                           n_runs=100)
trains = []
for i in range(100):
    train = Topyfic.read_train(f"train_{k}_{i}.p")
    trains.append(train)
    
main_train.combine_LDA_models(data, single_trains=trains)
main_train.save_train()

once you hace your train model, you can follow other tutorials for building top model and downstream analysis.

***if you need more infor about the inputs of each functions look at the API documentation [here](https://mortazavilab.github.io/Topyfic/html/api.html).***